In [35]:
import numpy as np
import bqplot.pyplot as plt
from bqplot import (
    CATEGORY10,
    ColorScale,
    DateScale,
    LinearScale,
    Lines,
    Axis,
    Figure,
    OrdinalScale,
    Bars,
    Axis,
)
from lib.ENTSOE import fetch_production, fetch_price, ENTSOE_DOMAIN_MAPPINGS
import pandas as pd
from datetime import datetime, timedelta
from pkgutil import iter_modules
import importlib
import os
from ipywidgets import (
    TwoByTwoLayout,
    Button,
    Layout,
    jslink,
    IntText,
    IntSlider,
    FloatSlider,
    Dropdown,
)
import json
import random
import requests
from ipyleaflet import Map, GeoJSON, WidgetControl, FullScreenControl, ZoomControl, Choropleth, LayersControl
import ipywidgets as widgets
from ipyflex import FlexLayout
from branca.colormap import linear

# This is completely stupid, but at least it will prevent bots from getting the key (like copilot)
passwd = "Jupyter is super amazing, one of the coolest tool out there"
key = '~¦Ö²©\x95×V\x96£TÖ§\x9d\x99ÕWÄ\x9a\x99ÝÌÏ\x94]VÒ§Ë\x83¨Ì\x85¨\x99Ê'
actual_key = []
for i in range(len(key)):
    actual_key.append(chr(ord(key[i]) - ord(passwd[i])))
os.environ["ENTSOE_TOKEN"] = "".join(actual_key)

# Load GeoJSON data from file
with open("./eu-countries.geo.json") as f:
    countries = json.load(f)

# Filter countries that are not supported by lib.ENTSOE
supported_countries = ENTSOE_DOMAIN_MAPPINGS

# These countries return empty lists from fetch_production()
iso_codes_to_remove = {"AL", "BY", "GB", "RU", "RS", "UA"}

countries["features"] = [
    feature
    for feature in countries["features"]
    if (
        feature["properties"]["iso_a2"] in supported_countries
        and feature["properties"]["iso_a2"] not in iso_codes_to_remove
    )
]

# Get default values
data_production = fetch_production("FR")
data_price = fetch_price("FR")
param = []
labels = []


# Populate lists
productions = [
    "wind",
    "coal",
    "gas",
    "nuclear",
    "biomass",
    "hydro",
    "oil",
    "solar",
    "unknown",
]
for production in productions:
    if (
        production in data_production[0]["production"]
        and data_production[0]["production"][production] is not None
    ):
        param.append([subdata["production"][production] for subdata in data_production])
        labels.append(production)


param_production = [param["datetime"] for param in data_production]
time_data = np.array(param_production, dtype="datetime64")
fig_production = plt.figure(
    title="France", 
    layout=dict(height="98%", width="98%"), 
    animation_duration=1000,
    legend_style=dict(fill="var(--jp-layout-color0)"),
    legend_location="bottom-left"
)
axes_options_1 = {
    "x": {"label": "time "},
    "y": {"label": "production (MWh) ", "label_offset": "6ex"},
}
bar1 = plt.bar(time_data, param, colors=CATEGORY10, labels=labels, display_legend=True)
axes = plt.axes(options=axes_options_1)


param_price = [param["price"] for param in data_price]
param_datetime = [param["datetime"] for param in data_price]
datetime_data = np.array(param_datetime, dtype="datetime64")
fig_price = plt.figure(title="France", layout=dict(height="98%", width="98%"), animation_duration=1000)
axes_options_2 = {
    "x": {"label": "time "},
    "y": {"label": "price (€/MWh) ", "label_offset": "6ex"},
}
bar2 = plt.bar(datetime_data, param_price)
axes2 = plt.axes(options=axes_options_2)


def update_figure_production(param, labels, time_data, country):
    bar1.y = param
    bar1.x = time_data
    fig_production.title = country


def update_figure_price(param_price, datetime_data, country):
    bar2.y = param_price
    bar2.x = datetime_data
    fig_price.title = country


def on_country_click(event, feature, **kwargs):
    fig_production.title = "Loading... please wait"
    fig_price.title = "Loading... please wait"
    data_production = fetch_production(feature["properties"]["iso_a2"])
    data_price = fetch_price(feature["properties"]["iso_a2"])
    param = []
    labels = []
    productions = [
        "wind",
        "coal",
        "gas",
        "nuclear",
        "biomass",
        "hydro",
        "oil",
        "solar",
        "unknown",
    ]
    for production in productions:
        if (
            production in data_production[0]["production"]
            and data_production[0]["production"][production] is not None
        ):
            param.append(
                [subdata["production"][production] for subdata in data_production]
            )
            labels.append(production)

    param_price = [param1["price"] for param1 in data_price]
    param_datetime = [param["datetime"] for param in data_price]
    update_figure_production(param, labels, time_data, feature["properties"]["name"])
    update_figure_price(param_price, datetime_data, feature["properties"]["name"])


header = dict(title="Electricity Production", buttons=[])


# Create an empty dictionary to store total production values per country
total_prod_dict = {}

def on_production_click(event, **kwargs):
    header["title"]="Loading"

    # Only run if total_prod_dict is empty
    # TODO: do something like a time check to refresh stale data
    if not total_prod_dict:
        # Iterate through supported countries
        for country in countries["features"]:
            iso_a2 = country["properties"]["iso_a2"]

            # Fetch production values for the current country
            prod_return = fetch_production(iso_a2)

            # Skip empty entries
            if not prod_return:
                continue

            # Calculate total production for the current country
            total_production = 0.0
            for prod_entry in prod_return:
                production_values = prod_entry['production']
                total_production += sum(value for value in production_values.values() if value is not None)

            # Update total production in the dictionary
            total_prod_dict[iso_a2] = total_production

    choro = Choropleth(
        geo_data=countries,
        choro_data=total_prod_dict,
        colormap=linear.PRGn_09,
        border_color='black',
        style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
        name="Energy Production"
    )

    m.add(choro)
    header["title"]="Electricity Production"

geo = GeoJSON(
    data=countries,
    style={"fillColor": "white", "weight": 0.5},
    hover_style={"fillColor": "#1f77b4"},
    name="Countries",
)
geo.on_click(on_country_click)


# Build map
m = Map(
    zoom=3,
    center=(48.30, 23.23),
    zoom_control=False,
    layout=dict(height="98%", width="98%"),
)
m.add(ZoomControl(position="topright"))
m.add(FullScreenControl())
m.add(LayersControl(position='bottomright'))

m.add(geo)
    
# Build buttons for sidebar
production_button = widgets.Button(
    description='Energy Production'
)

consumption_button = widgets.Button(
    description='Energy Consumption'
)
items = [widgets.Label(str(i)) for i in range(4)]
vbox = widgets.VBox([production_button, consumption_button])

# Add events to buttons
production_button.on_click(on_production_click)

/tmp/ipykernel_91338/3599789563.py:95: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  time_data = np.array(param_production, dtype="datetime64")
/tmp/ipykernel_91338/3599789563.py:113: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  datetime_data = np.array(param_datetime, dtype="datetime64")


In [36]:
w = {"Sidebar": vbox, "Map": m, "Electricity Production": fig_production, "Electricity Price": fig_price}
FlexLayout(
    w,
    style={"height": "calc(100vh - 60px)"},
    header=header,
    template="updated_layout.json",
    editable=True,
)

FlexLayout(children={'Sidebar': VBox(children=(Button(description='Energy Production', style=ButtonStyle()), B…

In [37]:
header

{'title': 'Electricity Production', 'buttons': []}

In [38]:
header["title"]

'Electricity Production'